# Notebook projet e-commerce

Initialisation et création de la base de donnée

In [5]:
import pandas as pd
import numpy as np
import sqlite3 as sqli
import os
from sqlalchemy import create_engine
from IPython.display import Markdown
from IPython.display import display, Latex
from faker import Faker

def md(input):
    display(Markdown(input))

#display all duplicated rows for all cols
def show_duplicated_rows(ds):
    dup_ds = ds[ds.duplicated()]
    if len(dup_ds) == 0:
        md("*Aucune ligne dupliquée*")
    else:
        display(dup_ds)
        
#display all duplicated rows for some cols only
def show_duplicated_cells(ds, cols):
    dup_ds = ds[ds.duplicated(subset = cols)]
    if len(dup_ds) == 0:
        md(f"{str(cols)} : Aucune valeur dupliquée")
    else:
        md(f"{str(cols)} : {len(dup_ds)}")

#from a dataframe get sql to create tables, file must be corrected with pk, fk and correct datatypes
def create_tables_sql_file(dataframes):
    query = ""
    for table_name, df in dataframes.items():
        query += pd.io.sql.get_schema(df, table_name)+";"
    with open("tables.sql", "w") as f:
        f.write(query)

#delete db file and create a new one with sqlite
def create_database(db_name):
    os.system(f"rm {db_name}.db")
    os.system(f"sqlite3 {db_name}.db < tables_with_keys.sql")

#insert dataframe data into db
def dataframe_to_insert(dataframes):
    conn = create_engine('sqlite:///olist.db')
    for table_name, dt in datasets.items():
        dt.to_sql(table_name, conn, if_exists="append", index = False)
        
    
csv_files = {
    "customers": "olist_customers_dataset.csv",
    "sellers" : "olist_sellers_dataset.csv",
    "products" : "olist_products_dataset.csv",
    "orders" : "olist_orders_dataset.csv",
    "order_items" : "olist_order_items_dataset.csv",
    "order_payments" : "olist_order_payments_dataset.csv",
    "order_reviews" : "olist_order_reviews_dataset.csv",
    "category_name_translation" : "product_category_name_translation.csv",
    "geolocation" : "olist_geolocation_dataset.csv",
}

fake = Faker("FR-fr")

csv_path = "csv/"

datasets = {}
for table_name, file_name in csv_files.items():
    datasets[table_name] = pd.read_csv(f"{csv_path}{file_name}")
    
create_tables_sql_file(datasets)
create_database("olist")
dataframe_to_insert(datasets)
    
print("✅")
print("Setup complete.")

✅
Setup complete.


## Customers

In [6]:
ds = datasets["customers"]
display(ds.head())
md(f"**Taille** : {'{:,}'.format(len(ds))}")
md("**Cellules vides**")
display(ds.isna().sum())
md("**Lignes dupliquées**")
show_duplicated_rows(ds)
md("**Cellules dupliquées**")
show_duplicated_cells(ds, ["customer_id"])
show_duplicated_cells(ds, ["customer_unique_id"])
md("**Problème**")
md("""
    - On utilise un second id pour simuler une table de jointure (1 client => plusieurs adresses)
    - Il manque plusieurs informations : nom, prenom, date de naissance, email, telephone
""")
md("**Solution**")
md("""
    - Créer une table *customer_addresses* et ne plus utiliser qu'un seul id (customer_unique_id qui deviendra customer_id)
    - Modifier les colonnes de la table *customer* pour compléter le profil
""")

md("**Update**")
ds.drop(columns = ["customer_id"], inplace = True)
ds.rename(columns = {'customer_unique_id':'customer_id'}, inplace = True)
datasets["customer_addresses"] = ds.copy()
size = len(datasets["customer_addresses"]["customer_id"].value_counts())
customers = {
    "customer_id" : datasets["customers"]["customer_id"].value_counts(),
    "first_name" : [fake.first_name() for _ in range(size)],
    "last_name" : [fake.last_name() for _ in range(size)],
    "email" : [fake.email() for _ in range(size)],
    "phone" : [fake.phone_number() for _ in range(size)],
    "birthdate" : [fake.date_between(start_date='-100y', end_date='-18y') for _ in range(size)]
}
datasets["customers"] = pd.DataFrame(data=customers)
md("***table customers***")
display(datasets["customers"].head())
md(f"**Taille** : {'{:,}'.format(len(datasets['customers']))}")
md("***table customer_addresses***")
display(datasets["customer_addresses"].head())

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


**Taille** : 99,441

**Cellules vides**

customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

**Lignes dupliquées**

...

***table customers***

,customer_id,first_name,last_name,email,phone,birthdate
8d50f5eadf50201ccdcedfb9e2ac8455,17,Victoire,Blondel,alexandreclerc@tiscali.fr,03 88 07 40 75,2001-06-19
3e43e6105506432c953e165fb2acf44c,9,Marianne,Torres,celina13@dbmail.com,0344706538,1948-02-06
6469f99c1f9dfae7733b25662e7f1782,7,Cécile,Pons,laurentlaurent@tele2.fr,+33 1 05 83 76 05,1938-04-21
1b6c7548a2a1f9037c1fd3ddfed95f33,7,Aimée,Gerard,mahewilliam@hotmail.fr,0804928962,1980-08-29
ca77025e7201e3b30c44b472ff346268,7,Denise,Guilbert,aleger@ifrance.com,+33 (0)6 42 29 38 40,1965-12-14


**Taille** : 96,096

***table customer_addresses***

,customer_id,customer_zip_code_prefix,customer_city,customer_state
0,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


## Sellers

In [3]:
ds = pd.read_csv(f"{csv_path}{csv_files['sellers']}")
display(ds.head())
md(f"**Taille** : {'{:,}'.format(len(ds))}")
md("**Cellules vides**")
display(ds.isna().sum())
md("**Lignes dupliquées**")
show_duplicated_rows(ds)
md("**Cellules dupliquées**")
show_duplicated_cells(ds, ["seller_id"])
md("**Problème**")
md("""
    - Il manque plusieurs informations : nom, presentation...
""")
md("**Solution**")
md("""
    - Ajouter des colonnes
""")
md("**Update**")
size = len(ds)
ds["seller_name"] = [fake.company() for _ in range(size)]
ds["seller_presentation"] = [fake.paragraph() for _ in range(size)]
display(ds.head())


,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


**Taille** : 3,095

**Cellules vides**

seller_id                 0
seller_zip_code_prefix    0
seller_city               0
seller_state              0
dtype: int64

**Lignes dupliquées**

...


    - Il manque plusieurs informations : nom, presentation...


**Solution**


    - Ajouter des colonnes


**Update**

,seller_id,seller_zip_code_prefix,seller_city,seller_state,seller_name,seller_presentation
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP,Gaillard Renault S.A.R.L.,Serrer justice champ venir interroger fusil. R...
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP,Gay Ledoux S.A.R.L.,Sauver soumettre appel position. Cinq regard p...
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ,Petitjean,Réveiller enfant malade garçon. Remercier natu...
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP,Charles SA,Jamais entre oncle puis pitié nez enfance. Ord...
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP,Pascal,Forêt billet affaire argent. Aider cou roche e...


## Products

In [44]:
ds = datasets["products"]
display(ds.head())
md(f"**Taille** : {'{:,}'.format(len(ds))}")
md("**Cellules vides**")
display(ds.isna().sum())
md("**Lignes dupliquées**")
show_duplicated_rows(ds)
md("**Cellules dupliquées**")
show_duplicated_cells(ds, ["product_id"])
md("**Problème**")
md("""
    - product_name_length, product_description_length et product_photos_qty sont des valeurs calculées on peut s'en passer
    - Il manque plusieurs informations : nom, description, photos...
""")
md("**Solution**")
md("""
    - Supprimer et ajouter les colonnes associées
    - Créer une table product_photo pour gérer les photos
""")
ds.drop(columns = ["product_name_lenght", "product_description_lenght", "product_photos_qty"], inplace = True)
size = len(ds)
ds["description"] = [fake.paragraph() for _ in range(size)]
ds["name"] = [f"product{i}" for i in range(size)]
md("**Update**")
md("***Table products***")
display(datasets["products"])
products_with_photos_number = 50
products_with_photos = np.random.choice(ds["product_id"], products_with_photos_number)
datasets["product_photos"] = pd.DataFrame.from_dict({
    "id" : [i for i in range(1,products_with_photos_number+1)],
    "product_id" : products_with_photos,
    "photo_path": [fake.file_path() for i in range(products_with_photos_number)]
})
md("***Table product photos***")
md("La table est simplifiée... un même produit peut avoir plusieurs photos")
datasets["product_photos"].head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


**Taille** : 32,951

**Cellules vides**

product_id                      0
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_photos_qty            610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64

**Lignes dupliquées**

...

***Table products***

,product_id,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm,description,name
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,225.0,16.0,10.0,14.0,Passion cheveu chez personne. Sembler rêver oc...,product0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,1000.0,30.0,18.0,20.0,Promettre remettre long occasion quel nerveux ...,product1
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,154.0,18.0,9.0,15.0,Amour foi troisième plaine. Honneur eau soin l...,product2
3,cef67bcfe19066a932b7673e239eb23d,bebes,371.0,26.0,4.0,26.0,Tromper lien frais appartement interroger obli...,product3
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,625.0,20.0,17.0,13.0,Pouvoir avenir cesse bien. Toute nerveux étroi...,product4
...,...,...,...,...,...,...,...,...
32946,a0b7d5a992ccda646f2d34e418fff5a0,moveis_decoracao,12300.0,40.0,40.0,40.0,Éteindre honte danger ni empêcher désigner. Mi...,product32946
32947,bf4538d88321d0fd4412a93c974510e6,construcao_ferramentas_iluminacao,1700.0,16.0,19.0,16.0,Contenir décrire habitant suffire. Plus annonc...,product32947
32948,9a7c6041fa9592d9d9ef6cfe62a71f8c,cama_mesa_banho,1400.0,27.0,7.0,27.0,Rencontrer assister prince crainte plaindre pa...,product32948
32949,83808703fc0706a22e264b9d75f04a2e,informatica_acessorios,700.0,31.0,13.0,20.0,User lieu français règle depuis épais.,product32949


***Table product photos***

La table est simplifiée... un même produit peut avoir plusieurs photos

,id,product_id,photo_path
0,1,abcde4527db854fbcf9ba5a0ebb571fe,/vouloir/envelopper.flac
1,2,b598f3e8cd6dedd51d454f6123a6e1e5,/du/noire.css
2,3,38ec77783936fa7f33d24dc56a46eca1,/classe/sonner.mp3
3,4,2f465f0f879ab88842042a85bac3a937,/recommencer/titre.docx
4,5,db59f0dfd5593c492b2ff1a6ab50cc2d,/lorsque/vieux.wav


## Orders

In [45]:
ds = datasets["orders"]
display(ds.head())
md(f"**Taille** : {'{:,}'.format(len(ds))}")
md("**Cellules vides**")
display(ds.isna().sum())
md("**Lignes dupliquées**")
show_duplicated_rows(ds)
md("**Cellules dupliquées**")
show_duplicated_cells(ds, ["order_id"])
md("**Valeurs de order_status**")
display(ds['order_status'].value_counts())
md("**Problème**")
md("""
    - Il manque le prix total de la commande
""")
md("**Solution**")
md("""
    - Ajouter le total
""")
md("**Update**")
md("à faire")

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


**Taille** : 99,441

**Cellules vides**

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

**Lignes dupliquées**

...


    - Il manque le prix total de la commande


**Solution**


    - Ajouter le total


**Update**

à faire

## Order items

In [47]:
ds = datasets["order_items"]
display(ds.head())
md(f"**Taille** : {'{:,}'.format(len(ds))}")
md("**Cellules vides**")
display(ds.isna().sum())
md("**Lignes dupliquées**")
show_duplicated_rows(ds)
md("**Cellules dupliquées**")
show_duplicated_cells(ds, ["order_id", "product_id", "seller_id"])
md("**Que fait la colonne order_item_id ?**")
display(ds[ds['order_id'] == '8272b63d03f5f79c56e9e4120aec44ef'])
md("**Problème**")
md("""
    - redondance : Comment le montre le tableau ci-dessous si la quantité du produit commandé est supérieure à 1, il y a autant de ligne identique qui sont crées dans la table (seule la colonne order_item_id est incrémentée)...
    - la table comporte trois clés étrangères ce qui crée aussi de la redondance et complexifie les requêtes (ex : trouver toutes les ventes pour un vendeur)
""")
md("**Solution**")
md("""
    - Supprimer la colonne *order_item_id* et utiliser une colonne *quantité* à la place
    - Créer une table order_seller qui fera le lien entre la table order et la table order-item
""")
md("**Update**")
md("Le code ci-dessous permet d'ajouter la colonne quantité à la place d'order_item_id mais il n'est pas performant")
# same_products = []
# def already_seen(row):
#     global same_products
#     merge_id = f"{row.order_id}/{row.product_id}"
#     if merge_id in same_products:
#         return False
#     else:
#         same_products.append(merge_id)
#         return True
    
# ds.shape
# ds.sort_values(by = "order_item_id", inplace = True, ascending = False)
# res = df.apply(already_seen, axis = 1)
# ds = ds[res]


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


**Taille** : 112,650

**Cellules vides**

order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64

**Lignes dupliquées**

...


    - redondance : Comment le montre le tableau ci-dessous si la quantité du produit commandée est supérieure à 1, il y a autant de ligne identique qui sont crées dans la table (seule la colonne order_item_id est incrémentée)...
    - la table comporte trois clés étrangères ce qui crée aussi de la redondance et complexifie les requêtes (ex : trouver toutes les ventes pour un vendeur)


**Solution**


    - Supprimer la colonne *order_item_id* et utiliser une colonne *quantité* à la place
    - Créer une table order_seller qui fera le lien entre la table order et la table order-item


**Update**

Le code ci-dessous permet d'ajouter la colonne quantité à la place d'order_item_id mais il n'est pas performant

## Order payments

In [38]:
ds = datasets["order_payments"]
display(ds.shape)
md(f"**Taille** : {'{:,}'.format(len(ds))}")
md("**Cellules vides**")
display(ds.isna().sum())
md("**Lignes dupliquées**")
show_duplicated_rows(ds)
md("**Cellules dupliquées**")
show_duplicated_cells(ds, ["order_id"])
md("**Valeurs de payment_type**")
display(ds['payment_type'].value_counts())
display(ds[ds['payment_type'] == "not_defined"])
display(ds[ds['payment_value'] == 0])
md("**Problème**")
md("""
    - Redondance : Si le client paie par bon d'achat, il y a autant de lignes que de bons d'achat utilisés incrémentant la colonne payment_sequential
    - Si le client paie en plusieurs fois (payment_installments) on ne garde pas de trace des différents paiements...
    - 3 paiements ont un type de paiement non défini et un montant de 0 on peut les supprimmer ainsi que 6 autres paiements par bon d'achat avec un montant de 0 (voir tableaux ci-dessus)
    - Il manque plusieurs informations telles que le statut du paiement et on a pas de traces des tentatives de paiement
""")
md("**Solution**")
md("""
    - Supprimer les paiements énoncés
    - Ajouter une table payment_traces (payment_id, payment_type, payment_value, status, created_at) et utiliser la table order_payments (payment_id, order_id, payment_installments), on a plus besoin de payment_value qui se calcule dans payment_traces
""")
ds.drop(ds[ds.payment_value == 0.0].index, inplace=True)
md("**Update**")
md("après supression des paiements invalides")
display(ds.shape)

(103877, 5)

**Taille** : 103,877

**Cellules vides**

order_id                0
payment_sequential      0
payment_type            0
payment_installments    0
payment_value           0
dtype: int64

**Lignes dupliquées**

...

**Solution**


    - Supprimer les paiements énoncés
    - Ajouter une table payment_traces (payment_id, payment_type, payment_value, status, created_at) et utiliser la table order_payments (payment_id, order_id, payment_installments), on a plus besoin de payment_value qui se calcule dans payment_traces


**Update**

après supression des paiements invalides

(103877, 5)

## Order reviews

In [48]:
ds = datasets["order_reviews"]
display(ds.head())
md(f"**Taille** : {'{:,}'.format(len(ds))}")
md("**Cellules vides**")
display(ds.isna().sum())
md("**Lignes dupliquées**")
show_duplicated_rows(ds)
md("**Cellules dupliquées**")
show_duplicated_cells(ds, ["review_id"])
show_duplicated_cells(ds, ["order_id"])
show_duplicated_cells(ds, ["review_id", "order_id"])
md("**Les reviews ne sont pas uniques...**")
display(ds['review_id'].value_counts())
display(ds[ds['review_id'] == "2172867fd5b1a55f98fe4608e1547b4b"])
md("**Problème**")
md("""
    - Il est possible d'utiliser la même évaluation pour plusieurs commandes alors que chaque éavluation devrait être indépendante. De plus on devrait pouvoir évaluer chaque vendeur pour chaque commande de manière indépendante.
    - Il y a beaucoup de valeurs vides pour le titre et la description, ce n'est pas un problème en soit car on a toujours le score mais ça peut être lié à un problème d'interface (peut être supprimer le champs titre car il est très peu utilisé).
    - pas de cols answer_message
""")
md("**Solution**")
md("""
    - L'ajout de la table order_seller va permettre de lier l'évaluation au vendeur avec la clé order_seller_id au lieu d'order_id 
    - rajout de la colonne answer_message
""")
md("**Update**")
md("A faire")

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


**Taille** : 100,000

**Cellules vides**

review_id                      0
order_id                       0
review_score                   0
review_comment_title       88285
review_comment_message     58247
review_creation_date           0
review_answer_timestamp        0
dtype: int64

**Lignes dupliquées**

...


    - Il est possible d'utiliser la même évaluation pour plusieurs commandes alors que chaque éavluation devrait être indépendante. De plus on devrait pouvoir évaluer chaque vendeur pour chaque commande de manière indépendante.
    - Il y a beaucoup de valeurs vides pour le titre et la description, ce n'est pas un problème en soit car on a toujours le score mais ça peut être lié à un problème d'interface (peut être supprimer le champs titre car il est très peu utilisé).
    - pas de cols answer_message


**Solution**


    - L'ajout de la table order_seller va permettre de lier l'évaluation au vendeur avec la clé order_seller_id au lieu d'order_id 
    - rajout de la colonne answer_message


**Update**

A faire

## Category name translation

In [49]:
ds = datasets["category_name_translation"]
display(ds.head())
md(f"**Taille** : {'{:,}'.format(len(ds))}")
md("**Cellules vides**")
display(ds.isna().sum())
md("**Lignes dupliquées**")
show_duplicated_rows(ds)
md("**Cellules dupliquées**")
show_duplicated_cells(ds, ["product_category_name"])
show_duplicated_cells(ds, ["product_category_name_english"])
md("**Problème**")
md("""
    - Si on décide un jour de changer le nom d'une cétagorie il faudra répéter la modification sur tous les produits liés
""")
md("**Solution**")
md("""
    - Ajout d'un index product_category_id dans la table
""")
md("**Update**")
md("A faire")

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor


**Taille** : 71

**Cellules vides**

product_category_name            0
product_category_name_english    0
dtype: int64

**Lignes dupliquées**

...


    - Si on décide un jour de changer le nom d'une cétagorie il faudra répéter la modification sur tous les produits liés


**Solution**


    - Ajout d'un index product_category_id dans la table


**Update**

A faire

## Geolocation

In [50]:
ds = datasets["geolocation"]
display(ds.head())
md(f"**Taille** : {'{:,}'.format(len(ds))}")
md("**Cellules vides**")
display(ds.isna().sum())
md("**Lignes dupliquées**")
show_duplicated_rows(ds)
md("**Cellules dupliquées**")
display(ds['geolocation_city'].value_counts())
show_duplicated_cells(ds, ["geolocation_zip_code_prefix","geolocation_lat", "geolocation_lng"])
md("**Problème**")
md("""
    - Pourquoi ne pas utiliser une API externe au lieu d'un million de lignes dans une table ?
    - Supprimer les doublons
""")
md("**Solution**")
md("""
    - Suppression de 261831 doublons.
""")
md("**Update**")
md("après supression des lignes dupliquées")
ds.drop_duplicates(["geolocation_zip_code_prefix","geolocation_lat", "geolocation_lng"], inplace = True)
ds.shape

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


**Taille** : 1,000,163

**Cellules vides**

geolocation_zip_code_prefix    0
geolocation_lat                0
geolocation_lng                0
geolocation_city               0
geolocation_state              0
dtype: int64

**Lignes dupliquées**

...

**Solution**


    - Suppression de 261831 doublons voir un peu plus si on prendait juste en compte les colonnes geolocation_zip_code_prefix, geolocation_lat et geolocation_lat mais ne sachant pas comment l'application va chercher ses données on s'abstiendra.


**Update**

après supression des lignes dupliquées

(720154, 5)